In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid3_u_27000_1/"

"../../graphs/grid3_u_27000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.699458 seconds (535.44 k allocations: 29.663 MB, 2.23% gc time)
  0.294562 seconds (539.82 k allocations: 12.916 MB)
  9.079306 seconds (86.79 M allocations: 2.194 GB, 4.72% gc time)
Working on iteration 10 with a-norm error 8307.401904429318
Working on iteration 20 with a-norm error 7237.7880139592435
Working on iteration 30 with a-norm error 6262.304345338127
Working on iteration 40 with a-norm error 5117.30124427074
Working on iteration 50 with a-norm error 4197.110496602085
Working on iteration 60 with a-norm error 3334.4387009691714
Working on iteration 70 with a-norm error 2716.831281717767
Working on iteration 80 with a-norm error 2181.4084998721773
Working on iteration 90 with a-norm error 1681.795888704747
Working on iteration 100 with a-norm error 1324.6791034442156
Working on iteration 110 with a-norm error 1028.3939324092735
Working on iteration 120 with a-norm error 818.6631066727875
Working on iteration 130 with a-norm error 665.3340491893445
Working on iteration 140 